In [ ]:
import pandas as pd
import matching

In [ ]:
df,shift_type_dict = matching.read_sheet("Shiftsplan_lux025.xlsx","ShiftList_KW16")

In [ ]:
shift_degree = [df[s].sum() for s in shift_type_dict]
len([x for x in shift_degree if x < 2])
for i,key in enumerate(shift_type_dict):
    if shift_degree[i] < 2:
        shift_type_dict[key] = []

In [ ]:
df = df.sample(frac=1).reset_index(drop=True)
base_mat = matching.shiftplan_to_adj_mat(df)
shift_type_dict_copy = shift_type_dict.copy()
for i in range(20):
    sl_mat = matching.make_sl_adj_mat(df,base_mat,shift_type_dict_copy)
    flow_sl = matching.get_flow_from_adj_mat(sl_mat)
    w_mat = matching.make_worker_adj_mat(df,base_mat,flow_sl,2)
    flow = matching.get_flow_from_adj_mat(w_mat)

    shift_list = matching.get_shift_list(df,flow_sl,flow)

    print(flow_sl.flow_value+flow.flow_value)
    
    for k in shift_list:
        if (shift_list[k]["lead"] is not None) and (shift_list[k] is None or len(shift_list[k]["worker"]) == 0):
            shift_type_dict_copy[k] = []
            shift_list = None
            break
    
    if shift_list is not None:
        break

w_mat_2 = matching.make_worker_adj_mat(df,w_mat,flow,2)
flow_2 = matching.get_flow_from_adj_mat(w_mat_2)
shift_list = matching.get_shift_list(df,flow_sl,flow,flow_2)
print(flow_sl.flow_value+flow.flow_value+flow_2.flow_value)

In [ ]:
for k in shift_list:
    print(k)
    print(shift_list[k]["lead"])
    for m in shift_list[k]["worker"]:
        print(m)
    print()

In [ ]:
matching.draw_matching(df,base_mat)

In [ ]:
sl_mat = matching.make_sl_adj_mat(df,base_mat,shift_type_dict)
matching.draw_matching(df,sl_mat)

In [ ]:
matching.draw_matching(df,flow_sl.flow)

In [ ]:
matching.draw_matching(df,flow.flow)